### Here we extract features from TMA patches (original + augmented)
Goal is to test MIL classifier, but code not working properly for model training part

--------

## Libraries

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
import random

import cv2
import timm
import pickle
from PIL import Image, ImageFile
from collections import defaultdict

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Data

In [6]:
combined_tma = pd.read_csv("/root/ubc_ocean/anar/augmented-tma/combined_tma.csv")

In [7]:
combined_tma['path'] = combined_tma['path'].str.replace('../', '')
combined_tma['path'] = combined_tma['path'].str.replace('./', '')
combined_tma['path'] = combined_tma['path'].str.replace('augmented-tma', 'anar/augmented-tma')
combined_tma

,Unnamed: 0,image_id,label,image_width,image_height,is_tma,path
0,0,91,HGSC,3388,3388,True,TMA_images/train_images/91.png
1,1,4134,MC,2964,2964,True,TMA_images/train_images/4134.png
2,2,8280,HGSC,2964,2964,True,TMA_images/train_images/8280.png
3,3,9200,MC,3388,3388,True,TMA_images/train_images/9200.png
4,4,13568,LGSC,2964,2964,True,TMA_images/train_images/13568.png
...,...,...,...,...,...,...,...
270,270,61797_6,HGSC,3388,3388,True,anar/augmented-tma/61797_6.png
271,271,61797_7,HGSC,3388,3388,True,anar/augmented-tma/61797_7.png
272,272,61797_8,HGSC,3388,3388,True,anar/augmented-tma/61797_8.png
273,273,61797_9,HGSC,3388,3388,True,anar/augmented-tma/61797_9.png


### Creating dictionary of feature vector bags

In [29]:
images_dir = '/root/ubc_ocean/'

# Increase the maximum number of pixels PIL can process
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load a pre-trained model for feature extraction
model = timm.create_model('resnet101', pretrained=True, num_classes=0)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to check if the tile has tissue present
def is_tissue_present(tile, area_threshold=0.55, low_saturation_threshold=20):   # 55% tissue
    hsv = cv2.cvtColor(tile, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    _, high_sat = cv2.threshold(s, low_saturation_threshold, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    tissue_mask = cv2.dilate(high_sat, kernel, iterations=2)
    tissue_mask = cv2.erode(tissue_mask, kernel, iterations=2)
    tissue_ratio = np.sum(tissue_mask > 0) / (tile_size * tile_size)
    return tissue_ratio > area_threshold

# Function to extract features from a tile
def extract_features(tile, model, transform):
    tile = Image.fromarray(tile)
    tile = transform(tile).unsqueeze(0)
    with torch.no_grad():
        features = model(tile)
    return features.squeeze(0).numpy()

# Function to process a patch of the image
def process_patch(patch, model, transform):
    if is_tissue_present(patch):
        features = extract_features(patch, model, transform)
        return features
    return None

# Define the size for the tiles
tile_size = 224

all_patch_features = {}

# Process each image, extract tiles, extract features, and store them
total_images = len(combined_tma)
for index, row in combined_tma.iterrows():
    tile_features = []  # List to hold the features for the current image

    # Get the image path
    image_name = row['path']  # Adjust based on your DataFrame structure
    image_path = os.path.join(images_dir, image_name)

    # Print the current status
    print(f"Processing image {index + 1} of {total_images}: {image_path}")

    try:
        with Image.open(image_path) as img:
            for y in range(0, img.height, tile_size):
                for x in range(0, img.width, tile_size):
                    # Read the patch
                    patch = img.crop((x, y, x + tile_size, y + tile_size))
                    patch = np.array(patch)  # Convert PIL Image to NumPy array

                    # Process the patch
                    features = process_patch(patch, model, transform)
                    if features is not None:
                        tile_features.append(features)

        # Store the extracted features and the label in the slide_features_part1 dictionary
        all_patch_features[image_name] = {
            'features': tile_features,
            'label': row['label']
        }

    except Exception as e:
        print(f"Error processing image {image_name}: {e}")

# Final save
with open('/root/ubc_ocean/anar/extracted-features/combinedtma_224px_resnet101_275.pkl', 'wb') as f:
    pickle.dump(all_patch_features, f)

Processing image 1 of 275: /root/ubc_ocean/TMA_images/train_images/91.png
Processing image 2 of 275: /root/ubc_ocean/TMA_images/train_images/4134.png
Processing image 3 of 275: /root/ubc_ocean/TMA_images/train_images/8280.png
Processing image 4 of 275: /root/ubc_ocean/TMA_images/train_images/9200.png
Processing image 5 of 275: /root/ubc_ocean/TMA_images/train_images/13568.png
Processing image 6 of 275: /root/ubc_ocean/TMA_images/train_images/17637.png
Processing image 7 of 275: /root/ubc_ocean/TMA_images/train_images/21020.png
Processing image 8 of 275: /root/ubc_ocean/TMA_images/train_images/29084.png
Processing image 9 of 275: /root/ubc_ocean/TMA_images/train_images/31594.png
Processing image 10 of 275: /root/ubc_ocean/TMA_images/train_images/35565.png
Processing image 11 of 275: /root/ubc_ocean/TMA_images/train_images/36302.png
Processing image 12 of 275: /root/ubc_ocean/TMA_images/train_images/36583.png
Processing image 13 of 275: /root/ubc_ocean/TMA_images/train_images/36783.png
P

In [8]:
with open('/root/ubc_ocean/anar/extracted-features/combinedtma_224px_resnet101_275.pkl', 'rb') as f:
    all_patch_features = pickle.load(f)
    
len(all_patch_features)

275

## Model Training (not works)
##### Anyway, for MIL classifier to perform well, having a diverse set is critical, augmenting the same image in 10 ways and extracting features would still result in relatively similar features.


In [38]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random

# Set a fixed random state for reproducibility
random_state = 33

# Convert slide_features to a suitable format
data = [(features['features'], features['label']) for path, features in all_patch_features.items()]

# Organize data by labels
data_by_label = defaultdict(list)
for features, label in data:
    data_by_label[label].append((features, label))

# Split data for each label into train, validation, and test
train_data = []
val_data = []
test_data = []

for label, label_data in data_by_label.items():
    # Split data for this label into train and test with a fixed random state
    train_val_label_data, test_label_data = train_test_split(label_data, test_size=0.10, random_state=random_state)
    
    # Split train data into train and validation with a fixed random state
    train_label_data, val_label_data = train_test_split(train_val_label_data, test_size=0.1, random_state=random_state)  # 0.25 x 0.8 = 0.2 of original
    
    # Append split data to respective sets
    train_data.extend(train_label_data)
    val_data.extend(val_label_data)
    test_data.extend(test_label_data)

# Shuffle the datasets
random.seed(random_state)
random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)

# Function to check balance in each set
def check_balance(dataset):
    label_counts = defaultdict(int)
    for _, label in dataset:
        label_counts[label] += 1
    return dict(label_counts)

# Display balance of each set
print("Train balance:", check_balance(train_data))
print("Validation balance:", check_balance(val_data))
print("Test balance:", check_balance(test_data))

Train balance: {'CC': 44, 'MC': 44, 'HGSC': 44, 'EC': 44, 'LGSC': 44}
Validation balance: {'HGSC': 5, 'MC': 5, 'LGSC': 5, 'CC': 5, 'EC': 5}
Test balance: {'LGSC': 6, 'CC': 6, 'EC': 6, 'HGSC': 6, 'MC': 6}


In [36]:
# Create a mapping from label strings to integers
unique_labels = sorted(set(label for _, label in data))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

class MILDataset(Dataset):
    def __init__(self, data, label_to_idx):
        self.data = data
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature_vectors, label = self.data[idx]
        label_idx = self.label_to_idx[label]  # Convert label to integer
        return torch.tensor(feature_vectors), torch.tensor(label_idx, dtype=torch.float32)

# Create Datasets for train, validation, and test
train_dataset = MILDataset(train_data, label_to_idx)
val_dataset = MILDataset(val_data, label_to_idx)
test_dataset = MILDataset(test_data, label_to_idx)

# Create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
import torch.nn as nn
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

class AttentionMIL(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(AttentionMIL, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Softmax(dim=0)
        )
        self.classifier = nn.Linear(hidden_dim, num_classes)  # num_classes instead of 1

    def forward(self, bag):
        h = torch.relu(self.fc1(bag))
        a = self.attention(h)
        v = torch.sum(a * h, dim=0)
        y = self.classifier(v)  # Remove softmax here; output raw scores
        return y, a

# Number of unique classes
num_classes = len(unique_labels)

model = AttentionMIL(input_dim=2048, hidden_dim=256, num_classes=num_classes)
loss_function = nn.CrossEntropyLoss()  # CrossEntropyLoss for multiclass
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Early Stopping Parameters
best_val_loss = float('inf')
patience = 4
patience_counter = 0

# Model Training with Validation
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # Training loop
    for bags, labels in train_loader:
        optimizer.zero_grad()
        bags = bags.squeeze(0)  # Remove the extra dimension from bags
        labels = labels.squeeze(0).long()  # Remove extra dimension and ensure long type for labels
        output, _ = model(bags)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(output.data, 0)
        train_total += 1
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for bags, labels in val_loader:
            bags = bags.squeeze(0)
            labels = labels.squeeze(0).long()
            output, _ = model(bags)
            loss = loss_function(output, labels)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 0)
            val_total += 1
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)
    
    # Step the scheduler
    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Stopping early due to no improvement in validation loss.")
        break

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for bags, labels in test_loader:
        output, _ = model(bags.squeeze(0))
        _, predicted_labels = torch.max(output, 0)  # Get the index of the max log-probability
        predictions.append(predicted_labels.item())  # Append scalar value
        true_labels.append(labels.squeeze(0).item())  # Append scalar value

# Convert lists to arrays for metric calculation
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro', zero_division=1)
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

# Use idx_to_label to map numeric predictions back to label names
predicted_labels = [idx_to_label[int(idx)] for idx in predictions]
true_label_names = [idx_to_label[int(idx)] for idx in true_labels]

# Now predicted_labels and true_label_names contain the label names
print(predicted_labels)
print(true_label_names)